In [ ]:
!pip install git+https://github.com/openai/swarm.git

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'PASTE YOUR OPENAI API KEY'

## Creating Agents and Handoffs Functions

In [ ]:
from swarm import Swarm, Agent

def handoff_to_weather_agent():
    """Transfer to the weather agent for weather queries."""
    print("Handing off to Weather Agent")
    return weather_agent

def handoff_to_math_agent():
    """Transfer to the math agent for mathematical queries."""
    print("Handing off to Math Agent")
    return math_agent

math_agent = Agent(
    name="Math Agent",
    instructions="You handle only mathematical queries.",
    functions = [handoff_to_weather_agent]
)

weather_agent = Agent(
    name="Weather Agent",
    instructions="You handle only weather-related queries.",
    functions = [handoff_to_math_agent]
)

In [ ]:
client = Swarm()
messages = [{"role": "user", "content": "What is 2+2?"}]
handoff_response = client.run(agent=weather_agent, messages=messages)
print(handoff_response)
print("=============")
print("=============")
print(handoff_response.messages[-1]["content"])

Handing off to Math Agent
messages=[{'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_lgi8IPE2s4yifgXLXAMiuxNP', 'function': {'arguments': '{}', 'name': 'handoff_to_math_agent'}, 'type': 'function'}], 'sender': 'Weather Agent'}, {'role': 'tool', 'tool_call_id': 'call_lgi8IPE2s4yifgXLXAMiuxNP', 'tool_name': 'handoff_to_math_agent', 'content': '{"assistant": "Math Agent"}'}, {'content': 'The answer to 2 + 2 is 4.', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None, 'sender': 'Math Agent'}] agent=Agent(name='Math Agent', model='gpt-4o', instructions='You handle only mathematical queries.', functions=[<function handoff_to_weather_agent at 0x7998740a7d00>], tool_choice=None, parallel_tool_calls=True) context_variables={}
The answer to 2 + 2 is 4.


In [ ]:
messages = [{"role": "user", "content": "How is weather in Canada in December?"}]
response = client.run(agent=math_agent, messages=messages)
print(response)
print("=============")
print("=============")
print(response.messages[-1]["content"])

Handing off to Weather Agent
messages=[{'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_CaQyJ9v8wh7YRVvRQrCvkh33', 'function': {'arguments': '{}', 'name': 'handoff_to_weather_agent'}, 'type': 'function'}], 'sender': 'Math Agent'}, {'role': 'tool', 'tool_call_id': 'call_CaQyJ9v8wh7YRVvRQrCvkh33', 'tool_name': 'handoff_to_weather_agent', 'content': '{"assistant": "Weather Agent"}'}, {'content': "I'm not currently set up to provide a direct response to your weather query, but I can refer you to a weather service for specific and up-to-date weather forecasts in Canada for December. Canada generally experiences cold weather during December, with varying conditions depending on the region. Coastal areas like Vancouver may have milder temperatures, while areas like Toronto, Ottawa, and Montreal see cooler, snowy conditions. Northern regions experience extreme cold. Checking a reliable weather service will provide the mos

## Creating RAG and NL2SQL Agent

In [ ]:
!pip install langchain langchain-chroma langchain-openai langchain-community pypdf sentence-transformers

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "/content/docs"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


Loaded 2 documents from the folder.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Split the documents into 4 chunks.


In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

from langchain_chroma import Chroma

collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke("Who was the founder of Futuresmart AI?")
print(retriever_results)

[Document(metadata={'page': 1, 'source': '/content/docs/FutureSmart AI .pdf'}, page_content='FutureSmart AI provides customized speech to text services, employing cutting -\nedge speech recognition technologies to cater to specific client needs. Ideal for \ncreating efficient documen tation and enabling voice -driven commands, this \nsolution boosts productivity and accessibility.'), Document(metadata={'page': 0, 'source': '/content/docs/FutureSmart AI .pdf'}, page_content='FutureSmart AI provides custom Natural Language Processing (NLP) \nsolutions for companies looking to get ahead of the future. Our \ndedicated team of Data Scientists and ML Engineers provides an end -\nto-end solution from data labeling to modeling and deploying an ML \nmodel tailored to your specific use case . \nFounder: Pradip Nichite  \n \nServices:  \nText Classification  \nAt FutureSmart AI, we develop custom text classificati on solutions using \nadvanced NLP techniques tailored to your specific business req

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


def retrieve_and_generate(question):
  template = """Answer the question based only on the following context:
  {context}
  Question: {question}
  Answer: """

  prompt = ChatPromptTemplate.from_template(template)

  def docs2str(docs):
      return "\n\n".join(doc.page_content for doc in docs)

  rag_chain = (
      {"context": retriever | docs2str, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )
  response = rag_chain.invoke(question)
  return response

question = "Who was the founder of Futuresmart AI?"
result = retrieve_and_generate(question)

print(f"Question: {question}")
print(f"Answer: {result}")

Question: Who was the founder of Futuresmart AI?
Answer: The founder of FutureSmart AI is Pradip Nichite.


## Setting up DB for NL2SQL

In [ ]:
!wget https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite

In [ ]:
!mv Chinook_Sqlite.sqlite Chinook.db

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from operator import itemgetter
import re
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def sql_response_gen(question):
  remove_code_block_syntax = lambda text: re.sub(r"```(sql|)\s*(.*?)\s*```", r"\2", text, flags=re.DOTALL)
  execute_query = QuerySQLDataBaseTool(db=db)
  write_query = create_sql_query_chain(llm, db)

  answer_prompt = PromptTemplate.from_template(
      """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

  Question: {question}
  SQL Query: {query}
  SQL Result: {result}
  Answer: """
  )

  chain = (
      RunnablePassthrough.assign(query=write_query | RunnableLambda(remove_code_block_syntax)).assign(
          result=itemgetter("query") | execute_query
      )
      | answer_prompt
      | llm
      | StrOutputParser()
  )

  response = chain.invoke({"question": question})
  return response

question = "How many employees are there?"
result = sql_response_gen(question)

print(f"Question: {question}")
print(f"Answer: {result}")

Question: How many employees are there?
Answer: There are 8 employees.


In [ ]:
from swarm import Swarm, Agent

rag_agent = Agent(
    name="RAG Agent",
    instructions="You retrieve relevant information from the company's knowledge base and generate responses to general queries about the company.",
    functions=[retrieve_and_generate]
)

nl2sql_agent = Agent(
    name="NL2SQL Agent",
    instructions="You handle database queries.",
    functions=[sql_response_gen]
)

In [ ]:
central_agent = Agent(
    name="Central Agent",
    instructions="Determine if the query is about general company information (RAG) or a database query (NL2SQL), and route the query accordingly."
)

# Define handoff functions to delegate tasks to the correct agent
def transfer_to_nl2sql():
    """Transfer the task to the NL2SQL Agent for database queries."""
    return nl2sql_agent

def transfer_to_rag():
    """Transfer the task to the RAG Agent for general company queries."""
    return rag_agent

# Attach the handoff functions to the central agent
central_agent.functions = [transfer_to_nl2sql, transfer_to_rag]

In [ ]:
client = Swarm()

# Example 1: Asking about the company
print("\n--- Example 1: Asking about the company ---")
messages = [{"role": "user", "content": "What does Futuresmart AI offer?"}]
response = client.run(agent=central_agent, messages=messages)
if isinstance(response, Agent):
    selected_agent = response
    result = selected_agent.functions
    print(result)
else:
    print(response.messages[-1]["content"])


# Example 2: Asking from the SQL DB
print("\n--- Example 2: Asking from the SQL DB ---")
messages = [{"role": "user", "content": "How many employees are there in the database?"}]
response = client.run(agent=central_agent, messages=messages)
if isinstance(response, Agent):
    selected_agent = response
    result = selected_agent.functions
    print(result)
else:
    print(response.messages[-1]["content"])



--- Example 1: Asking about the company ---
FutureSmart AI offers a range of services including customized speech-to-text services, Natural Language Processing (NLP) solutions, text classification, and the creation of custom chatbots. These services are designed to enhance productivity, accessibility, decision-making processes, and operational efficiency through advanced technologies and tailored solutions.

--- Example 2: Asking from the SQL DB ---
There are 8 employees in the database.
